📘 Notebook – Classificação da Qualidade de Bananas

📌 1. Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

📌 2. Carregamento da base de dados

In [ ]:
df = pd.read_csv('banana_quality.csv')

df.head()

📌 3. Estatísticas gerais

In [ ]:
print("\nEstatísticas Descritivas")
df.describe().T

📌 4. Transformação da coluna Quality → 0/1

In [ ]:
le = LabelEncoder()
df['Quality01'] = le.fit_transform(df['Quality'])
nomeclasse = le.classes_

print("Qualidade codificada como: 0 = Bad, 1 = Good")

📌 5. Remoção de linhas com valores fora do intervalo (-4, 4)

In [ ]:
nmrs = df.select_dtypes(include=np.number).columns.tolist()

mask = (df[nmrs] > -4).all(axis=1) & (df[nmrs] < 4).all(axis=1)

df_filtered = df[mask].copy()

linhas_removidas = len(df) - len(df_filtered)
df = df_filtered

print("Linhas antes:", len(df) + linhas_removidas)
print("Linhas após :", len(df))
print("Removidas   :", linhas_removidas)

📌 6. Separação em treino, validação e teste

In [ ]:
X = df.drop(['Quality', 'Quality01'], axis=1)
y = df['Quality01']

X_train_valid, X_test, y_train_valid, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, y_train_valid, test_size=0.25, random_state=42, stratify=y_train_valid
)

print("Total: ", len(X))
print("Treino: ", len(X_train))
print("Validação: ", len(X_valid))
print("Teste: ", len(X_test))

📌 7. Normalização

In [ ]:
scaler = StandardScaler()

train_scaled = scaler.fit_transform(X_train)
valid_scaled = scaler.transform(X_valid)
test_scaled = scaler.transform(X_test)

📌 8. Gráfico

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(x='Quality', data=df, color='yellow')
plt.title('Qualidade das Bananas')
plt.show()

📌 9. Treinamento do modelo Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(train_scaled, y_train)

y_valid_pred = rf.predict(valid_scaled)

📌 10. Avaliação do modelo

In [ ]:
mat_conf = confusion_matrix(y_valid, y_valid_pred)
acc = accuracy_score(y_valid, y_valid_pred)

print("Acurácia:", acc)

plt.figure(figsize=(8, 6))
sns.heatmap(mat_conf, annot=True, fmt='d', cmap='inferno',
            xticklabels=nomeclasse, yticklabels=nomeclasse)
plt.title('Matriz de Confusão')
plt.ylabel('Real')
plt.xlabel('Predição')
plt.show()

📌 11. Predição de um exemplo

In [ ]:
exemplo = X_test.iloc[[0]]
y_real = y_test.iloc[0]

exemplo_scaled = scaler.transform(exemplo)
pred = rf.predict(exemplo_scaled)[0]

print("\nAmostra:")
display(pd.DataFrame(exemplo_scaled, columns=X_test.columns, index=['Amostra']))

print("\nQualidade Predita:", nomeclasse[pred])
print("Qualidade Real:", nomeclasse[y_real])